In [1]:
import open3d as o3d
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Multiway registration
Multiway registration is the process of aligning multiple pieces of geometry in a global space. Typically, the input is a set of geometries (e.g., point clouds or RGBD images) $\{\mathbf{P}_{i}\}$. The output is a set of rigid transformations $\{\mathbf{T}_{i}\}$, so that the transformed point clouds $\{\mathbf{T}_{i}\mathbf{P}_{i}\}$ are aligned in the global space.

Open3D implements multiway registration via pose graph optimization. The backend implements the technique presented in [\[Choi2015\]](../reference.html#choi2015).

## Input
The first part of the tutorial code reads three point clouds from files. The point clouds are downsampled and visualized together. They are misaligned.

In [2]:
def load_point_clouds(voxel_size=0.0):
    pcds = []
    for i in range(2):
        pcd = o3d.io.read_point_cloud("PointClouds/Lab_Cam%d.ply" %
                                      (i+1))
        pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
        pcds.append(pcd_down)
    return pcds

In [3]:
voxel_size = 0.007
pcds_down = load_point_clouds(voxel_size)
#o3d.visualization.draw_geometries(pcds_down,
#                                  zoom=0.3412,
#                                  front=[0.4257, -0.2125, -0.8795],
#                                  lookat=[2.6172, 2.0475, 1.532],
#                                  up=[-0.0694, -0.9768, 0.2024])

## Pose graph
A pose graph has two key elements: nodes and edges. A node is a piece of geometry $\mathbf{P}_{i}$ associated with a pose matrix $\mathbf{T}_{i}$ which transforms $\mathbf{P}_{i}$ into the global space. The set $\{\mathbf{T}_{i}\}$ are the unknown variables to be optimized. `PoseGraph.nodes` is a list of `PoseGraphNode`. We set the global space to be the space of $\mathbf{P}_{0}$. Thus $\mathbf{T}_{0}$ is the identity matrix. The other pose matrices are initialized by accumulating transformation between neighboring nodes. The neighboring nodes usually have large overlap and can be registered with [Point-to-plane ICP](../pipelines/icp_registration.ipynb#point-to-plane-ICP).

A pose graph edge connects two nodes (pieces of geometry) that overlap. Each edge contains a transformation matrix $\mathbf{T}_{i,j}$ that aligns the source geometry $\mathbf{P}_{i}$ to the target geometry $\mathbf{P}_{j}$. This tutorial uses [Point-to-plane ICP](../pipelines/icp_registration.ipynb#point-to-plane-ICP) to estimate the transformation. In more complicated cases, this pairwise registration problem should be solved via [Global registration](global_registration.ipynb).

[\[Choi2015\]](../reference.html#choi2015) has observed that pairwise registration is error-prone. False pairwise alignments can outnumber correctly aligned pairs. Thus, they partition pose graph edges into two classes. **Odometry edges** connect temporally close, neighboring nodes. A local registration algorithm such as ICP can reliably align them. **Loop closure edges** connect any non-neighboring nodes. The alignment is found by global registration and is less reliable. In Open3D, these two classes of edges are distinguished by the `uncertain` parameter in the initializer of `PoseGraphEdge`.

In addition to the transformation matrix $\mathbf{T}_{i}$, the user can set an information matrix $\mathbf{\Lambda}_{i}$ for each edge. If $\mathbf{\Lambda}_{i}$ is set using function `get_information_matrix_from_point_clouds`, the loss on this pose graph edge approximates the RMSE of the corresponding sets between the two nodes, with a line process weight. Refer to Eq (3) to (9) in [\[Choi2015\]](../reference.html#choi2015) and [the Redwood registration benchmark](http://redwood-data.org/indoor/registration.html) for details.

The script creates a pose graph with three nodes and three edges. Among the edges, two of them are odometry edges (`uncertain = False`) and one is a loop closure edge (`uncertain = True`).

In [6]:
def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=50))
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

In [7]:
print("Full registration ...")
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)

Full registration ...
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.4133, RMSE 0.0604
[Open3D DEBUG] Residual : 2.50e-03 (# of elements : 166953)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.5060, RMSE 0.0583
[Open3D DEBUG] Residual : 2.38e-03 (# of elements : 204372)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.5747, RMSE 0.0566
[Open3D DEBUG] Residual : 2.22e-03 (# of elements : 232151)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.6238, RMSE 0.0563
[Open3D DEBUG] Residual : 2.13e-03 (# of elements : 251989)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.6625, RMSE 0.0554
[Open3D DEBUG] Residual : 2.02e-03 (# of elements : 267597)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.6997, RMSE 0.0551
[Open3D DEBUG] Residual : 1.98e-03 (# of elements : 282635)
[Open3D DEBUG] ICP Iteration #6: Fitness 0.7325, RMSE 0.0542
[Open3D DEBUG] Residual : 1.89e-03 (# of elements : 295902)
[Open3D DEBUG] ICP Iteration #7: Fitness 0.7610, RMSE 0.0534
[Open3D DEBUG] Residual : 1.81e-03 (# of elem

[Open3D DEBUG] ICP Iteration #18: Fitness 0.6156, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 248665)
[Open3D DEBUG] ICP Iteration #19: Fitness 0.6160, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 248807)
[Open3D DEBUG] ICP Iteration #20: Fitness 0.6163, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 248948)
[Open3D DEBUG] ICP Iteration #21: Fitness 0.6164, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 248989)
[Open3D DEBUG] ICP Iteration #22: Fitness 0.6165, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 249044)
[Open3D DEBUG] ICP Iteration #23: Fitness 0.6168, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 249129)
[Open3D DEBUG] ICP Iteration #24: Fitness 0.6167, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 249093)
[Open3D DEBUG] ICP Iteration #25: Fitness 0.6168, RMSE 0.0050
[Open3D DEBUG] Residual : 1.24e-05 (# of elements : 249138)
[Open3D DEBUG] ICP Itera

Open3D uses the function `global_optimization` to perform pose graph optimization. Two types of optimization methods can be chosen: `GlobalOptimizationGaussNewton` or `GlobalOptimizationLevenbergMarquardt`. The latter is recommended since it has better convergence property. Class `GlobalOptimizationConvergenceCriteria` can be used to set the maximum number of iterations and various optimization parameters.

Class `GlobalOptimizationOption` defines a couple of options. `max_correspondence_distance` decides the correspondence threshold. `edge_prune_threshold` is a threshold for pruning outlier edges. `reference_node` is the node id that is considered to be the global space.

In [8]:
print("Optimizing PoseGraph ...")
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)

Optimizing PoseGraph ...
[Open3D DEBUG] Validating PoseGraph - finished.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 27.476505
[Open3D DEBUG] [Initial     ] residual : 1.333235e-28, lambda : 6.104590e+00
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 27.476505
[Open3D DEBUG] [Initial     ] residual : 1.333235e-28, lambda : 6.104590e+00
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] CompensateReferencePoseGraphNode : reference : 0


The global optimization performs twice on the pose graph. The first pass optimizes poses for the original pose graph taking all edges into account and does its best to distinguish false alignments among uncertain edges. These false alignments have small line process weights, and they are pruned after the first pass. The second pass runs without them and produces a tight global alignment. In this example, all the edges are considered as true alignments, hence the second pass terminates immediately.

## Visualize optimization
The transformed point clouds are listed and visualized using `draw_geometries`.

In [9]:
print("Transform points and display")
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
#o3d.visualization.draw_geometries(pcds_down,
#                                  zoom=0.3412,
#                                  front=[0.4257, -0.2125, -0.8795],
#                                  lookat=[2.6172, 2.0475, 1.532],
#                                  up=[-0.0694, -0.9768, 0.2024])

Transform points and display
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 0.97430518  0.2215535   0.04053966  0.00774666]
 [-0.2218662   0.97507144  0.00332751 -0.0050339 ]
 [-0.03879185 -0.01223639  0.99917239  0.26420299]
 [ 0.          0.          0.          1.        ]]


## Make a combined point cloud
`PointCloud` has a convenience operator `+` that can merge two point clouds into a single one. In the code below, the points are uniformly resampled using `voxel_down_sample` after merging. This is recommended post-processing after merging point clouds since it can relieve duplicated or over-densified points.

In [10]:
pcds = load_point_clouds(voxel_size)
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
#pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud("PointClouds/Lab_Cam1_Cam2_multiway_registration_0.007.ply", pcd_combined)
#o3d.visualization.draw_geometries([pcd_combined_down],
#                                  zoom=0.3412,
#                                  front=[0.4257, -0.2125, -0.8795],
#                                  lookat=[2.6172, 2.0475, 1.532],
#                                  up=[-0.0694, -0.9768, 0.2024])

True

<div class="alert alert-info">
    
**Note:**

Although this tutorial demonstrates multiway registration for point clouds, the same procedure can be applied to RGBD images. See [Make fragments](../reconstruction_system/make_fragments.rst) for an example.

</div>

In [10]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd_combined, depth=12)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 859049 / 858699
[Open3D DEBUG] #   Got kernel density: 5.785 (s), 1298.54 (MB) / 1298.54 (MB) / 3906 (MB)
[Open3D DEBUG] #     Got normal field: 5.051 (s), 1564.33 (MB) / 1564.33 (MB) / 3906 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.712400e-06 / 3.189134e+00
[Open3D DEBUG] #       Finalized tree: 7.297 (s), 1772.46 (MB) / 1772.46 (MB) / 3906 (MB)
[Open3D DEBUG] #  Set FEM constraints: 10.686 (s), 1545.45 (MB) / 1772.46 (MB) / 3906 (MB)
[Open3D DEBUG] #Set point constraints: 2.579 (s), 1395.71 (MB) / 1772.46 (MB) / 3906 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 23368612 / 9099448 / 17607537
[Open3D DEBUG] Memory Usage: 1395.730 MB
[Open3D DEBUG] # Linear system solved: 19.383 (s), 1619.43 (MB) / 1772.46 (MB) / 3906 (MB)
[Open3D DEBUG] Got average: 1.00100 (s), 1471.07 (MB) / 1772.46 (MB) / 3906 (MB)
[Open3D DEBUG] Iso-Value: 5.019360e-01 = 4.311876e+05 / 8.590490e+05
[Open3D DEBUG] # 

In [11]:
print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

visualize densities


In [12]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

remove low density vertices
TriangleMesh with 2427689 points and 4851284 triangles.


In [13]:
o3d.io.write_triangle_mesh("PointClouds/Lab_Cam1_Cam2_multiway_registration_0.007_mesh_orientedNormals.ply", mesh)

True